## Purpose - To scrape data on authors, their affiliation and reported addresses at the level of each paper

### Import Packages

In [ ]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException

In [ ]:
EXE_PATH = r"F:\Winter 2020\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(executable_path=EXE_PATH)

In [ ]:
base = "https://www.nber.org/papers?page=1&perPage=100&sortBy=public_date"
browser.get(base)
browser.implicitly_wait(10)

text = browser.find_element_by_class_name("search__info-results").text
pages = int(text[text.find('F') + 2 : text.find('R') - 1]) + 1
pages = math.ceil(pages/100)
pages

In [ ]:
#df = pd.read_excel(r'F:\Winter 2020\NBER_WP.xlsx')
#scraped = list(df['paper'])
scraped = []

In [ ]:
dic = {}
count = 0

errors = []

url = "https://www.nber.org/papers?page={q}&perPage=100&sortBy=public_date"

for i in range(1, pages + 1, 1):
    
    print(i)
    
    browser.get(url.format(q = str(i)))
    browser.implicitly_wait(30)
    
    paper_links = []
    for element in browser.find_elements_by_xpath('//a[contains(@href,"/papers/")]'):
        paper_link = element.get_attribute('href')
        paper_links.append(paper_link)
    
    paper_links = set(paper_links)
    for paper_link in paper_links:
        
        try:
            html = urlopen(paper_link)
            soup = BeautifulSoup(html, 'lxml')
        
            paper_name = soup.find('h1', class_ = 'page-header__title').text.strip()
        
            if paper_name not in scraped:
                dic[count] = {'paper': paper_name}
                print(paper_name)
                
                paper_month = soup.find('time').text
                dic[count]['paper month'] = paper_month
                
                temp = soup.find('div', class_ = 'page-header__citation-item').text.strip()
                dic[count]['paper number'] = temp[temp.find('Paper') + 6:]
                
                for each in soup.find_all('div', class_ = 'info-grid__item'):
                    text = [t.text for t in each.find_all('a')]
                    l = len(text)
    
                    if 'topics' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper topic ' + str(c + 1)] = text[c]
                    elif 'programs' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper program ' + str(c + 1)] = text[c]
                    elif 'working groups' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper group ' + str(c + 1)] = text[c]
                    elif 'projects' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper project ' + str(c + 1)] = text[c]
        
                authors = soup.find_all('div', class_ = 'page-header__authors')[0].find_all('a')
        
                num = 1
                for author in authors:
            
                    author_name = author.text
                    dic[count]['author ' + str(num)] = author_name
            
                    author_link = "https://www.nber.org" + author.get('href')
                    html = urlopen(author_link)
                    soup = BeautifulSoup(html, 'lxml')
            
                    author_position = soup.find_all('div', class_ = 'person-header__title')[0].text.strip()
                    dic[count]['author position ' + str(num)] = author_position
            
                    author_affiliation = soup.find_all('div', class_ = 'person-header__summary')[0].text.strip()
                    dic[count]['author affiliation ' + str(num)] = author_affiliation
                    
                    author_address = soup.find_all('div', 'contact-card__social')[-1].text.strip().replace('\n', ' ')
                    dic[count]['author address ' + str(num)] = author_address
            
                    num+=1
            
                count += 1 
        except:
            errors.append(paper_link)

In [ ]:
df=pd.DataFrame.from_dict(dic).T
df.to_excel(r'F:\Winter 2020\NBER_WP.xlsx',engine='xlsxwriter')